In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [215]:
import numpy as np
import pandas as pd
import xarray as xr
import xdatasets as xd
from sklearn.cluster import HDBSCAN, OPTICS, AgglomerativeClustering
from sklearn.decomposition import PCA

import xhydro as xh
import xhydro.frequency_analysis as xhfa
import xhydro.gis as xhgis

ModuleNotFoundError: No module named 'xhfa'

In [3]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

ds

<xarray.Dataset> Size: 574kB
Dimensions:        (id: 5, time: 20454)
Coordinates: (12/15)
    drainage_area  (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date       (id) datetime64[ns] 40B 2006-10-13 2024-06-02 ... 1996-08-13
  * id             (id) object 40B '020302' '020404' '020502' '020602' '020802'
    latitude       (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude      (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name           (id) object 40B 'Saint' 'York' ... 'Dartmouth' 'Madeleine'
    ...             ...
    spatial_agg    <U9 36B 'watershed'
    start_date     (id) datetime64[ns] 40B 1989-08-12 1980-10-01 ... 1970-01-01
  * time           (time) datetime64[ns] 164kB 1970-01-01 ... 2025-12-31
    time_agg       <U4 16B 'mean'
    timestep       <U1 4B 'D'
    variable       <U10 40B 'streamflow'
Data variables:
    streamflow     (id, time) float32 409kB nan nan nan nan ... nan nan nan nan

In [10]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "annual": {},
}

In [11]:
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

ds_4fa

<xarray.Dataset> Size: 3kB
Dimensions:                (id: 5, time: 56)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_annual  (id, time) float32 1kB nan nan nan ... nan nan nan
Attributes:
    cat:variable:          ('streamflow_max_spring',)
    cat:xrfreq:            YS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [12]:
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)
ds_4fa

<xarray.Dataset> Size: 8kB
Dimensions:                (id: 5, time: 56)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
Data variables:
    streamflow_max_spring  (id, time) float32 1kB nan nan nan ... nan nan nan
    streamflow_max_annual  (id, time) float32 1kB nan nan nan ... nan nan nan
    volume_sum_spring      (id, time) float64 2kB nan nan nan ... nan nan nan
    volume_sum_annual      (id, time) float64 2kB nan nan nan ... nan nan nan
Attributes:
    cat:variable:          ('streamflow_max_spring',)
    cat:xrfreq:            YS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [13]:
# Extraction des SHP pour les staton hydrométriques provinciales
gdf = xd.Query(
    **{
        "datasets": {
            "deh_polygons": {
                "id": ["020*"],
                "regulated": ["Natural"],
                "variables": ["streamflow"],
            }
        },
        "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
    }
).data.reset_index()
gdf

,Station,Superficie,geometry
0,020302,1071.505249,"POLYGON ((-65.54653 48.91282, -65.54639 48.912..."
1,020404,664.096924,"POLYGON ((-65.1477 49.05904, -65.14748 49.0589..."
2,020502,57.292057,"POLYGON ((-64.45703 48.9948, -64.45692 48.9946..."
3,020602,626.996155,"POLYGON ((-64.97292 49.17614, -64.97283 49.176..."
4,020802,1184.306641,"POLYGON ((-65.26495 49.21157, -65.26473 49.211..."


In [39]:
dswp = xhgis.watershed_properties(
    gdf[["Station", "geometry"]], unique_id="Station", output_format="xarray"
)
cent = dswp["centroid"].to_numpy()
lon = [ele[0] for ele in cent]
lat = [ele[1] for ele in cent]
dswp = dswp.assign(lon=("Station", lon))
dswp = dswp.assign(lat=("Station", lat))
dswp = dswp.drop("centroid")
dswp

<xarray.Dataset> Size: 240B
Dimensions:    (Station: 5)
Coordinates:
  * Station    (Station) object 40B '020302' '020404' '020502' '020602' '020802'
Data variables:
    area       (Station) float64 40B 1.072e+09 6.641e+08 ... 6.27e+08 1.184e+09
    perimeter  (Station) float64 40B 3.637e+05 2.496e+05 ... 2.067e+05 3.478e+05
    gravelius  (Station) float64 40B 3.134 2.732 1.747 2.329 2.851
    lon        (Station) float64 40B -65.09 -65.24 -64.46 -64.91 -65.55
    lat        (Station) float64 40B 48.74 48.9 48.96 49.04 49.02

In [163]:
data, pca = xhfa.regional.fit_pca(dswp, n_components=3)
data

<xarray.DataArray (Station: 5, components: 3)> Size: 120B
array([[ 1.99288321, -1.57005881,  0.22668637],
       [ 0.43621688, -0.14911349, -0.54138187],
       [-3.3311851 , -0.43277391, -0.01838273],
       [-0.90217836,  0.78713613,  0.32433607],
       [ 1.80426337,  1.36481008,  0.00874216]])
Coordinates:
  * Station     (Station) object 40B '020302' '020404' ... '020602' '020802'
  * components  (components) int32 12B 0 1 2

In [168]:
data.to_dataframe(name="value").reset_index().pivot(
    index="Station", columns="components"
).corr()

value                            
components                   0             1             2
      components                                          
value 0           1.000000e+00  8.896369e-17  5.999128e-16
      1           8.896369e-17  1.000000e+00  8.403164e-16
      2           5.999128e-16  8.403164e-16  1.000000e+00

In [170]:
pca.n_components_

3

In [ ]:
xhfa.regional.get_group_from_fit(HDBSCAN, {"min_cluster_size": 2}, data)

In [ ]:
xhfa.regional.get_group_from_fit(OPTICS, {"min_samples": 2}, data)

In [233]:
groupes = xhfa.regional.get_group_from_fit(
    AgglomerativeClustering, {"n_clusters": 3}, data
)
groupes

[array(['020404', '020602', '020802'], dtype=object),
 array(['020502'], dtype=object),
 array(['020302'], dtype=object)]

In [231]:
from xhydro.frequency_analysis.regional import moment_l_vector

In [232]:
ds_moment = xr.apply_ufunc(
    moment_l_vector, ds_4fa, input_core_dims=[["time"]], output_core_dims=[["lmom"]]
).assign_coords(lmom=["l1", "l2", "l3", "tau", "tau3", "tau4"])
ds_moment

<xarray.Dataset> Size: 2kB
Dimensions:                (id: 5, lmom: 6)
Coordinates: (12/15)
    drainage_area          (id) float32 20B 1.09e+03 647.0 59.8 626.0 1.2e+03
    end_date               (id) datetime64[ns] 40B 2006-10-13 ... 1996-08-13
  * id                     (id) object 40B '020302' '020404' ... '020802'
    latitude               (id) float32 20B 48.77 48.81 48.98 48.98 49.2
    longitude              (id) float32 20B -64.52 -64.92 -64.43 -64.7 -65.29
    name                   (id) object 40B 'Saint' 'York' ... 'Madeleine'
    ...                     ...
    spatial_agg            <U9 36B 'watershed'
    start_date             (id) datetime64[ns] 40B 1989-08-12 ... 1970-01-01
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
  * lmom                   (lmom) <U4 96B 'l1' 'l2' 'l3' 'tau' 'tau3' 'tau4'
Data variables:
    streamflow_max_spring  (id, lmom) float64 240B 318.3 182.7 ... 0.2896 0.3008
    streamflow_max_annual  (id, lmom) float64 240B 396.2 104.8 ... 0.2926 0.2847
    volume_sum_spring      (id, lmom) float64 240B 337.9 56.86 ... 0.1953
    volume_sum_annual      (id, lmom) float64 240B 1.132e+03 170.1 ... 0.2247

In [235]:
ds_groups = xr.concat(
    [
        ds_4fa.sel(id=groupes[i]).assign_coords(group_id=i).expand_dims("group_id")
        for i in range(len(groupes))
    ],
    dim="group_id",
)
ds_moments_groups = xr.concat(
    [
        ds_moment.sel(id=groupes[i]).assign_coords(group_id=i).expand_dims("group_id")
        for i in range(len(groupes))
    ],
    dim="group_id",
)

In [236]:
ds_groups

<xarray.Dataset> Size: 22kB
Dimensions:                (group_id: 3, id: 5, time: 56)
Coordinates: (12/16)
  * id                     (id) object 40B '020302' '020404' ... '020802'
  * time                   (time) datetime64[ns] 448B 1970-01-01 ... 2025-01-01
  * group_id               (group_id) int32 12B 0 1 2
    drainage_area          (group_id, id) float32 60B nan 647.0 nan ... nan nan
    end_date               (group_id, id) datetime64[ns] 120B NaT ... NaT
    latitude               (group_id, id) float32 60B nan 48.81 nan ... nan nan
    ...                     ...
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    start_date             (group_id, id) datetime64[ns] 120B NaT ... NaT
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
Data variables:
    streamflow_max_spring  (group_id, id, time) float32 3kB nan nan ... nan nan
    streamflow_max_annual  (group_id, id, time) float32 3kB nan nan ... nan nan
    volume_sum_spring      (group_id, id, time) float64 7kB nan nan ... nan nan
    volume_sum_annual      (group_id, id, time) float64 7kB nan nan ... nan nan
Attributes:
    cat:variable:          ('streamflow_max_spring',)
    cat:xrfreq:            YS-JAN
    cat:frequency:         yr
    cat:processing_level:  indicators
    cat:id:

In [237]:
ds_moments_groups

<xarray.Dataset> Size: 4kB
Dimensions:                (group_id: 3, id: 5, lmom: 6)
Coordinates: (12/16)
  * id                     (id) object 40B '020302' '020404' ... '020802'
  * lmom                   (lmom) <U4 96B 'l1' 'l2' 'l3' 'tau' 'tau3' 'tau4'
  * group_id               (group_id) int32 12B 0 1 2
    drainage_area          (group_id, id) float32 60B nan 647.0 nan ... nan nan
    end_date               (group_id, id) datetime64[ns] 120B NaT ... NaT
    latitude               (group_id, id) float32 60B nan 48.81 nan ... nan nan
    ...                     ...
    source                 <U102 408B 'Ministère de l’Environnement, de la Lu...
    spatial_agg            <U9 36B 'watershed'
    start_date             (group_id, id) datetime64[ns] 120B NaT ... NaT
    time_agg               <U4 16B 'mean'
    timestep               <U1 4B 'D'
    variable               <U10 40B 'streamflow'
Data variables:
    streamflow_max_spring  (group_id, id, lmom) float64 720B nan nan ... nan nan
    streamflow_max_annual  (group_id, id, lmom) float64 720B nan nan ... nan nan
    volume_sum_spring      (group_id, id, lmom) float64 720B nan nan ... nan nan
    volume_sum_annual      (group_id, id, lmom) float64 720B nan nan ... nan nan